In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from modules.loader import ImmothepDataLoader
loader = ImmothepDataLoader()
del(ImmothepDataLoader)

from modules.csvLoader import csvLoader

DATASET_PATH = '../datas/RAW/'

%load_ext autoreload
%autoreload 2

## Téléchargement des données

In [2]:
loader.ensure_data_loaded()
del(loader)

Les fichiers sont correctement téléchargés


## Lecture des csv

In [3]:
valeurs_foncieres = pd.read_csv(DATASET_PATH + "valeursfoncieres-2019.txt", sep='|', encoding='utf-8')
del(DATASET_PATH)

# Observations sur le dataframe

## Les types de biens
```
Appartement
Dépendance
Maison
nan
Local industriel. commercial ou assimilé

nb local :	 2535791  soit	 100  % des locaux
nb appart :	 430975  soit	 17  % des locaux
nb maison :	 515398  soit	 20  % des locaux
nb dependance :	 312480  soit	 12  % des locaux
nb none_local :	 1182459  soit	 47  % des locaux
nb industriel :	 94479  soit	 4  % des locaux
```


In [4]:
for local in valeurs_foncieres['Type local'].drop_duplicates().values:
    print(local)
del(local)
print()
# appartement = valeurs_foncieres[valeurs_foncieres['Type local'] == 'Appartement']
maison = valeurs_foncieres[valeurs_foncieres['Type local'] == 'Maison']
# dependance = valeurs_foncieres[valeurs_foncieres['Type local'] == 'Dépendance']
# none_local = valeurs_foncieres[valeurs_foncieres['Type local'].isna()]
# industriel = valeurs_foncieres[valeurs_foncieres['Type local'] == 'Local industriel. commercial ou assimilé']

print('nb local :\t', len(valeurs_foncieres), '\tsoit\t', round(len(valeurs_foncieres)/len(valeurs_foncieres)*100), ' % des locaux' )
# print('nb appart :\t', len(appartement), '\tsoit\t', round(len(appartement)/len(valeurs_foncieres)*100), ' % des locaux' )
print('nb maison :\t', len(maison), '\tsoit\t', round(len(maison)/len(valeurs_foncieres)*100), ' % des locaux' )
# print('nb dependance :\t', len(dependance), '\tsoit\t', round(len(dependance)/len(valeurs_foncieres)*100), ' % des locaux' )
# print('nb none_local :\t', len(none_local), '\tsoit\t', round(len(none_local)/len(valeurs_foncieres)*100), ' % des locaux' )
# print('nb industriel :\t', len(industriel), '\tsoit\t', round(len(industriel)/len(valeurs_foncieres)*100), ' % des locaux' )

del(valeurs_foncieres)

Appartement
Dépendance
Maison
nan
Local industriel. commercial ou assimilé

nb local :	 2535791 	soit	 100  % des locaux
nb maison :	 515398 	soit	 20  % des locaux


## Filtres sur les maisons

Aucune de ces maisons n'est souhaitée dans les données du modèle IA
- nb maison non bati 	 10 	soit	 0.0  % des maisons
- nb maison sans terrain 	 22207 	soit	 4.3  % des maisons
- nb maison sans valeur 	 1419 	soit	 0.3  % des maisons
- nb maison sans piece 	 10 	soit	 0.0  % des maisons
- nb maison sans code postal 	 45 	soit	 0.0  % des maisons
- maisons_filtre == 491748

In [5]:
maison_non_bati = maison[maison['Surface reelle bati'].isna()]
maison_non_terrain = maison[maison['Surface terrain'].isna()]
maison_non_valeur = maison[maison['Valeur fonciere'].isna()]
maison_non_pieces = maison[maison['Nombre pieces principales'].isna()]
maison_non_postal = maison[maison['Code postal'].isna()]

print('nb maison non bati \t', len(maison_non_bati), '\tsoit\t', round(len(maison_non_bati)/len(maison)*100, 1), ' % des maisons' )
print('nb maison sans terrain \t', len(maison_non_terrain), '\tsoit\t', round(len(maison_non_terrain)/len(maison)*100, 1), ' % des maisons' )
print('nb maison sans valeur \t', len(maison_non_valeur), '\tsoit\t', round(len(maison_non_valeur)/len(maison)*100, 1), ' % des maisons' )
print('nb maison sans piece \t', len(maison_non_pieces), '\tsoit\t', round(len(maison_non_pieces)/len(maison)*100, 1), ' % des maisons' )
print('nb maison sans code postal \t', len(maison_non_postal), '\tsoit\t', round(len(maison_non_postal)/len(maison)*100, 1), ' % des maisons' )

del(maison_non_bati,maison_non_terrain,maison_non_valeur,maison_non_pieces,maison_non_postal)

maison_filtre = maison.dropna(subset=['Surface reelle bati','Surface terrain', 'Valeur fonciere', 'Nombre pieces principales', 'Code postal'])
del(maison)

print('maison_filtre ==', len(maison_filtre))

nb maison non bati 	 10 	soit	 0.0  % des maisons
nb maison sans terrain 	 22207 	soit	 4.3  % des maisons
nb maison sans valeur 	 1419 	soit	 0.3  % des maisons
nb maison sans piece 	 10 	soit	 0.0  % des maisons
nb maison sans code postal 	 45 	soit	 0.0  % des maisons
maison_filtre == 491748


## Observation du contenu des colonnes

Dans un objectif de performance, est-il nécessaire de toutes les conserver ?


In [6]:
liste_colonne_vide = []
print('Suppression des colonnes vides\n')

for col in maison_filtre.columns:
    if len(maison_filtre[maison_filtre[col].isna()]) == len(maison_filtre):
        print('Colonne ', col)
        maison_filtre = maison_filtre.drop(columns=col)

print('\nSuppression effectuée !')
del(liste_colonne_vide, col)

Suppression des colonnes vides

Colonne  Code service CH
Colonne  Reference document
Colonne  1 Articles CGI
Colonne  2 Articles CGI
Colonne  3 Articles CGI
Colonne  4 Articles CGI
Colonne  5 Articles CGI
Colonne  No Volume
Colonne  2eme lot
Colonne  Surface Carrez du 2eme lot
Colonne  3eme lot
Colonne  Surface Carrez du 3eme lot
Colonne  4eme lot
Colonne  Surface Carrez du 4eme lot
Colonne  5eme lot
Colonne  Surface Carrez du 5eme lot
Colonne  Identifiant local

Suppression effectuée !


In [7]:
print('Pourcentage valeur non vide par colonne')
for col in maison_filtre.columns:
    print( col, ' :\t', 100 - len(maison_filtre[maison_filtre[col].isna()]) / len(maison_filtre) * 100, ' %')

Pourcentage valeur non vide par colonne
No disposition  :	 100.0  %
Date mutation  :	 100.0  %
Nature mutation  :	 100.0  %
Valeur fonciere  :	 100.0  %
No voie  :	 99.63761926840577  %
B/T/Q  :	 7.106078723248487  %
Type de voie  :	 80.96565720653669  %
Code voie  :	 100.0  %
Voie  :	 99.99247582094894  %
Code postal  :	 100.0  %
Commune  :	 100.0  %
Code departement  :	 100.0  %
Code commune  :	 100.0  %
Prefixe de section  :	 3.76819021124642  %
Section  :	 99.99694965714146  %
No plan  :	 100.0  %
1er lot  :	 0.27798791250803845  %
Surface Carrez du 1er lot  :	 0.19176488770671085  %
Nombre de lots  :	 100.0  %
Code type local  :	 100.0  %
Type local  :	 100.0  %
Surface reelle bati  :	 100.0  %
Nombre pieces principales  :	 100.0  %
Nature culture  :	 100.0  %
Nature culture speciale  :	 3.278508504355898  %
Surface terrain  :	 100.0  %


In [10]:
csvLoader.save(maison_filtre, 'maison_filtre')

Successfully created the directory ../datas/CURATED/ 
